# 03 - Pruning Estructural y Cuantización

En este notebook aplicaremos pruning a las cabezas de atención y cuantización post-entrenamiento.

In [1]:
!pip install transformers datasets torch

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.3.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pyarrow-20.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached xxhash-3.5.0-cp312-cp312-manylinux_2_17_x86_64.ma

In [2]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import torch.nn.utils.prune as prune

# Cargar modelo fine-tuneado
model = AutoModelForSequenceClassification.from_pretrained('../outputs/models/finetuned')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/home/cleber/Escritorio/PC3_NLP/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '../outputs/models/finetuned'. Use `repo_type` argument if needed.

In [ ]:
# Pruning estructural de cabezas de atención
PRUNE_HEADS = {  # Ejemplo: podar cabezas en capas 1 y 2
    1: [0, 1],
    2: [2, 3]
}
model.distilbert.prune_heads(PRUNE_HEADS)
torch.save(model.state_dict(), '../outputs/models/pruned/model_pruned.pt')

In [ ]:
# Evaluar modelo podado
dataset = load_dataset('glue', 'mrpc')
def preprocess(batch):
    return tokenizer(batch['sentence1'], batch['sentence2'], truncation=True, padding='max_length')
dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
model.eval()
acc = 0
total = 0
for batch in dataset['validation']:
    inputs = {k: v.unsqueeze(0) for k, v in batch.items() if k in ['input_ids','attention_mask']}
    labels = batch['label'].unsqueeze(0)
    with torch.no_grad():
        outputs = model(**inputs)
    preds = outputs.logits.argmax(dim=-1)
    acc += (preds == labels).sum().item()
    total += labels.size(0)
print(f'Accuracy modelo podado: {acc/total:.4f}')

In [ ]:
# Cuantización dinámica post-entrenamiento
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
torch.save(quantized_model.state_dict(), '../outputs/models/quantized/model_quantized.pt')

In [ ]:
# Evaluar modelo cuantizado
quantized_model.eval()
acc_q = 0
total_q = 0
for batch in dataset['validation']:
    inputs = {k: v.unsqueeze(0) for k, v in batch.items() if k in ['input_ids','attention_mask']}
    labels = batch['label'].unsqueeze(0)
    with torch.no_grad():
        outputs = quantized_model(**inputs)
    preds = outputs.logits.argmax(dim=-1)
    acc_q += (preds == labels).sum().item()
    total_q += labels.size(0)
print(f'Accuracy modelo cuantizado: {acc_q/total_q:.4f}')

In [ ]:
# Comparativa de tamaños
def size_mb(path): return os.path.getsize(path)/1e6
original = size_mb('../outputs/models/finetuned/pytorch_model.bin')
pruned = size_mb('../outputs/models/pruned/model_pruned.pt')
quant = size_mb('../outputs/models/quantized/model_quantized.pt')
print(f'Original: {original:.2f} MB')
print(f'Pruned: {pruned:.2f} MB')
print(f'Quantized: {quant:.2f} MB')

In [ ]:
# Export a ONNX
from transformers import pipeline
onnx_path = '../outputs/models/quantized/model_quantized.onnx'
dummy_input = ('This is a test sentence.', 'Another sentence.')
nlp = pipeline('text-classification', model=quantized_model, tokenizer=tokenizer)
nlp.model.eval()
torch.onnx.export(
    nlp.model,
    (torch.tensor([tokenizer.encode(dummy_input[0], dummy_input[1])]),),
    onnx_path,
    opset_version=11
)